# Rearranging columns

In this problem, we have a table with several columns describing GO terms, however most part of them are NaN. So we want to create one single column with all the annotations associated to each protein sequence (row).


First, let's check our file and see the best way to open it:

In [1]:
import pandas as pd
import re

Now that we found the best way to read: skipping the first rows and the last ons (EOF), we save into variable df of data frame.

In [2]:
df = pd.read_excel('proteínas glândula aranha.xltx', skiprows=[0,1,2], skip_footer=1)
df.tail(2)

,#,Visible?,Starred?,Identified Proteins (632),Accession Number,Molecular Weight,Protein Grouping Ambiguity,Quantitative Variance,Taxonomy,biological adhesion,...,molecular function,molecular transducer activity,motor activity,nutrient reservoir activity,protein tag,structural molecule activity,transcription regulator activity,translation regulator activity,transporter activity,Mudpit_DATA.TXT (F003448 TUB NCBInr)
672,528.0,True,Empty,Putative tritil protein [Rhipicephalus pulchel...,gi|427777533,119 kDa,True,NaN,Rhipicephalus pulchellus,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
673,529.0,True,Empty,gi|375150834-DECOY,gi|375150834-DECOY,?,NaN,NaN,unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [3]:
df.columns

Index(['#', 'Visible?', 'Starred?', 'Identified Proteins (632)',
       'Accession Number', 'Molecular Weight', 'Protein Grouping Ambiguity',
       'Quantitative Variance', 'Taxonomy', 'biological adhesion',
       'biological regulation', 'cell killing', 'cellular process',
       'developmental process', 'establishment of localization', 'growth',
       'immune system process', 'localization', 'locomotion',
       'metabolic process', 'multi-organism process',
       'multicellular organismal process', 'pigmentation', 'reproduction',
       'reproductive process', 'response to stimulus', 'rhythmic process',
       'viral reproduction', 'Golgi apparatus', 'cytoplasm', 'cytoskeleton',
       'endoplasmic reticulum', 'endosome', 'extracellular region',
       'intracellular organelle', 'membrane', 'mitochondrion', 'nucleus',
       'organelle membrane', 'organelle part', 'plasma membrane', 'ribosome',
       'antioxidant activity', 'auxiliary transport protein activity',
       'bindin

All this columns will be part of only 3 columns:

- **Biological Process**: biological adhesion, biological regulation, cell killing, cellular process, developmental process, establishment of localization, growth, immune system process, localization, locomotion, metabolic process, multi-organism process, multicellular organismal process, response to stimulus, rhythmic process, antioxidant activity, auxiliary transport protein activity, binding, catalytic activity, chemorepellent activity, electron carrier activity, enzyme regulator activity, molecular transducer activity, motor activity, nutrient reservoir activity, structural molecule activity, transcription regulator activity, translation regulator activity, transporter activity, 

- **Cellular component**: Golgi apparatus, cytoplasm, cytoskeleton, endoplasmic reticulum, endosome, extracellular region, intracellular organelle, membrane, mitochondrion, nucleus, organelle membrane, organelle part, plasma membrane, ribosome 

- **Molecular Function**: molecular function

So, we create a dictionary for those 3 groups:

In [4]:
categories = {}
categories['biological process'] = """biological adhesion, biological regulation, cell killing, cellular process, 
    developmental process, establishment of localization, growth, immune system process, 
    localization, locomotion, metabolic process, multi-organism process, multicellular organismal process, 
    response to stimulus, rhythmic process, antioxidant activity, auxiliary transport protein activity, 
    binding, catalytic activity, chemorepellent activity, electron carrier activity, enzyme regulator activity, 
    molecular transducer activity, motor activity, nutrient reservoir activity, structural molecule activity, 
    transcription regulator activity, translation regulator activity, transporter activity"""

categories['cellular component'] = """Golgi apparatus, cytoplasm, cytoskeleton, endoplasmic reticulum, endosome, 
    extracellular region, intracellular organelle, membrane, mitochondrion, nucleus, organelle membrane, organelle part, 
    plasma membrane, ribosome"""

categories['molecular function'] = ['molecular function'] 
categories = {cat:re.split(',\s+', categories[cat]) if isinstance(categories[cat], str) else categories[cat] for cat in categories}

In [5]:
from itertools import chain
set(df.columns) - set(chain(*categories.values()))

{'#',
 'Accession Number',
 'Identified Proteins (632)',
 'Molecular Weight',
 'Mudpit_DATA.TXT (F003448 TUB NCBInr)',
 'Protein Grouping Ambiguity',
 'Quantitative Variance',
 'Starred?',
 'Taxonomy',
 'Visible?',
 'chaperone regulator activity',
 'chemoattractant activity',
 'metallochaperone activity',
 'pigmentation',
 'protein tag',
 'reproduction',
 'reproductive process',
 'viral reproduction'}

However, some columns (after **Visible?**) are not part of any group. So we need to insert in one of them, let say  **"biological process"**:

In [6]:
missing_subcategories = [    
    'chaperone regulator activity',
    'chemoattractant activity',
    'metallochaperone activity',
    'pigmentation',
    'protein tag',
    'reproduction',
    'reproductive process',
    'transporter activity',
    'viral reproduction'
]

categories['biological process'] = categories['biological process'] + missing_subcategories

In [7]:
set(df.columns) - set(chain(*categories.values()))

{'#',
 'Accession Number',
 'Identified Proteins (632)',
 'Molecular Weight',
 'Mudpit_DATA.TXT (F003448 TUB NCBInr)',
 'Protein Grouping Ambiguity',
 'Quantitative Variance',
 'Starred?',
 'Taxonomy',
 'Visible?'}

We still have some columns outside of the groups, let's save them to the end.

In [8]:
list(df.columns[:9]) + [df.columns[-1]] 

['#',
 'Visible?',
 'Starred?',
 'Identified Proteins (632)',
 'Accession Number',
 'Molecular Weight',
 'Protein Grouping Ambiguity',
 'Quantitative Variance',
 'Taxonomy',
 'Mudpit_DATA.TXT (F003448 TUB NCBInr)']

In [9]:
outras_colunas = list(df.columns[:9]) + [df.columns[-1]] 

Just to see how is one of the groups:

In [10]:
df[categories['biological process']].head()

,biological adhesion,biological regulation,cell killing,cellular process,developmental process,establishment of localization,growth,immune system process,localization,locomotion,...,transporter activity,chaperone regulator activity,chemoattractant activity,metallochaperone activity,pigmentation,protein tag,reproduction,reproductive process,transporter activity,viral reproduction
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df[categories['biological process']].notnull().sum().head(15)

biological adhesion                 0
biological regulation               2
cell killing                        0
cellular process                    9
developmental process               0
establishment of localization       5
growth                              0
immune system process               0
localization                        5
locomotion                          0
metabolic process                   3
multi-organism process              0
multicellular organismal process    0
response to stimulus                1
rhythmic process                    0
dtype: int64

To concatenate, we have three different functions!

In [12]:
def concatenar(lista, sep=';'):
    return sep.join(x for x in set(lista) if x != '')

df[categories['biological process']].fillna('').apply(concatenar).head()

biological adhesion                                                       
biological regulation    Wnt receptor signaling pathway;regulation of t...
cell killing                                                              
cellular process         exocytosis;methionine biosynthetic process;for...
developmental process                                                     
dtype: object

The concatenation was made in the rows instead of the columns! Why?

Look at the help of *apply* function and you will see that *apply* is set to work in the columns by default (axis=0), if you want to work in the rows, use *axis=1*. 

A better example (where the rows are not only NaN):

In [13]:
exemploNaoNulo = df['cellular process'].notnull()
dfSlice = df[exemploNaoNulo][categories['biological process']]
dfSlice.head()

,biological adhesion,biological regulation,cell killing,cellular process,developmental process,establishment of localization,growth,immune system process,localization,locomotion,...,transporter activity,chaperone regulator activity,chemoattractant activity,metallochaperone activity,pigmentation,protein tag,reproduction,reproductive process,transporter activity,viral reproduction
65,NaN,NaN,NaN,exocytosis,NaN,exocytosis,NaN,NaN,exocytosis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,exocytosis,NaN,exocytosis,NaN,NaN,exocytosis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,NaN,Wnt receptor signaling pathway,NaN,Wnt receptor signaling pathway,NaN,NaN,NaN,NaN,NaN,NaN,...,lipid transporter activity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lipid transporter activity,NaN
99,NaN,NaN,NaN,methionine biosynthetic process,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,NaN,NaN,NaN,AMP metabolic process,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Our concatenate functions in action:

In [14]:
dfSlice.fillna('').apply(concatenar, axis=1).head()

65                                            exocytosis
67                                            exocytosis
72     Wnt receptor signaling pathway;lipid transport...
99     purine nucleotide biosynthetic process;methion...
271    adenylate kinase activity;ATP binding;AMP meta...
dtype: object

Now we can create the final table e save it into csv/excel file!

First the columns that are the same as the original, than our special 3 groups!

In [15]:
df_concat = df[outras_colunas].copy()
df_concat.head()

,#,Visible?,Starred?,Identified Proteins (632),Accession Number,Molecular Weight,Protein Grouping Ambiguity,Quantitative Variance,Taxonomy,Mudpit_DATA.TXT (F003448 TUB NCBInr)
0,1.0,True,Empty,Cluster of PREDICTED: CAD protein [Tribolium c...,gi|91090153 [4],244 kDa,True,NaN,Tribolium castaneum,11
1,1.1,True,Empty,PREDICTED: CAD protein [Tribolium castaneum],gi|91090153,244 kDa,True,NaN,Tribolium castaneum,3
2,1.2,True,Empty,"CAD(r), partial [Bembidion planatum]",gi|382936963 (+1),32 kDa,True,NaN,Bembidion planatum,4
3,1.3,True,Empty,"CAD(r), partial [Bembidion proprium]",gi|382936975,33 kDa,True,NaN,Bembidion proprium,4
4,1.4,True,Empty,"CAD(r), partial [Bembidion xanthacrum]",gi|382937095,32 kDa,True,NaN,Bembidion xanthacrum,2


The following functions are the same, but using the other concatenate functions:

In [16]:
df_concat = df[outras_colunas].copy()
for cat in categories:
    df_concat[cat] = df[categories[cat]].fillna('').apply(concatenar, axis=1)
df_concat.to_excel('tabela_concatenada.xlsx', index=None)

# Hooray! Now we have a more concise table!!!